In [2]:
import numpy as np
from PIL import Image
import argparse
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_avaliable() else 'cpu')

AttributeError: module 'torch.cuda' has no attribute 'is_avaliable'

In [1]:
batch_size = 32
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

mnist_data = torchvision.datasets.MNIST('./mnist_data', train=True, download=True,
                                       transform=transforms)
dataloader = torch.utils.data.DataLoader(dataset=mnist_data, batch_size=batch_size, shuffle=True)

NameError: name 'transforms' is not defined

In [4]:
plt.imshow(next(iter(dataloader))[0][5][0], cmap=plt.cm.gray)
image_size = 28 * 28
hidden_size = 256
# 对抗器
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid()
)
# 生成器
latent_size = 64
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

D = D.to(device)
G = G.to(device)

loss_fn = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr = 0.002)
g_optimizer = torch.optim.Adam(G.parameters(), lr = 0.002)

total_steps = len(dataloader)
num_epochs = 30
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(dataloader):
        batch_size = images.shape[0]
        images = images.reshape(batch_size, image_size).to(device)
        
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        outputs = D(images)
        d_loss_real = loss_fn(outputs, real_labels)
        real_score = outputs # 对于D来说，越大越好
        
        # 开始生成fake image
        z = torch.randn(batch_size, latent_size).to(device)# latent variable
        fake_images = G(z)
        outputs= D(fake_images.detach())
        d_loss_fake = loss_fn(outputs, fake_labels)
        fake_score = outputs # 对于D来说，越小越好
        
        # 开始优化 discriminator
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # 开始优化generator
        outputs = D(fake_images)
        g_loss = loss_fn(outputs, real_labels)
        
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        
        g_loss.backward()
        g_optimizer.step()
        
        if i % 200 == 0:
            print('Epoch [{}/{}], step[{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x):{:.2f}, D(G(z)): {:.2f}'
                 .format(epoch, num_epochs, i, total_step, d_loss.item(), g_loss.item(), real_score.mean().item(),
                        fake_score.mean().item()))

NameError: name 'dataloader' is not defined

In [ ]:
z = torch.randn(batch_size, latent_size).to(device)# latent variable
fake_images = G(z)
fake_images = fake_images.view(batch_size, 28, 28).data.cpu().numpy()
plt.imshow(fake_images[0], cmap=plt.cm.gray)